### Example: Historical + SSP1-2.6 / SSP2-4.5 Monte Carlo simulation with OSCAR
### If you want to know and try OSCAR, you can see the documents in github https://github.com/tgasser/OSCAR
### This script is based on the example run scripts provided with the OSCAR model.

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt

from core_fct.mod_process import OSCAR
from core_fct.fct_loadP import load_all_param
from core_fct.fct_genMC import generate_config
from run_scripts.get_SSP_drivers import For_hist, For_scen

In [ ]:
# ==================================================
# 1. Load parameters and generate Monte Carlo ensemble
# ==================================================

Par0 = load_all_param(mod_region="RCP_5reg")
Par  = generate_config(Par0, nMC=200)


In [ ]:
# ==================================================
# 2. Historical simulation
# ==================================================

# Merge time-independent drivers into parameters
Par = xr.merge([
    Par,
    For_hist.drop_vars([v for v in For_hist if "year" in For_hist[v].dims])
])

# Keep only time-dependent historical forcings
For_hist = For_hist.drop_vars([v for v in For_hist if "year" not in For_hist[v].dims])

# Run historical period
Out_hist = OSCAR(Ini=None, Par=Par, For=For_hist)

In [ ]:
# ==================================================
# 3. Scenario simulation (SSP126 & SSP245 only)
# ==================================================

# Select scenarios and years
scen_keep = ["SSP126", "SSP245"]
For_scen = (
    For_scen
    .sel(year=slice(None, 2100))
    .sel(scen=scen_keep)
)

# Initial state = last year of historical run
Ini = Out_hist.isel(year=-1, drop=True)

# Variables to keep in output
var_keep = (
    ["D_Eluc", "D_Focean", "D_Fland", "D_Epf"]
    + ["tau_CH4", "tau_N2O"]
    + [v for v in OSCAR._processes if "RF" in v]
)

# Run scenarios
Out_scen = OSCAR(
    Ini=Ini,
    Par=Par,
    For=For_scen,
    var_keep=var_keep,
    nt=4,
)

In [ ]:
# ==================================================
# 4. Simple plotting utilities
# ==================================================

def plot_global(VAR):
    """Plot global mean time series with Monte Carlo spread."""
    plt.figure(figsize=(6, 4))

    # Historical
    if VAR in Out_hist:
        mu = Out_hist[VAR].mean("config")
        sd = Out_hist[VAR].std("config")
        plt.plot(Out_hist.year, mu, color="k", lw=2, label="Historical")
        plt.fill_between(Out_hist.year, mu - sd, mu + sd, color="k", alpha=0.3)

    # Scenarios
    for scen in Out_scen.scen.values:
        mu = Out_scen[VAR].sel(scen=scen).mean("config")
        sd = Out_scen[VAR].sel(scen=scen).std("config")
        plt.plot(Out_scen.year, mu, lw=2, label=scen)
        plt.fill_between(Out_scen.year, mu - sd, mu + sd, alpha=0.3)

    plt.xlabel("Year")
    plt.ylabel(f"{VAR} ({Out_scen[VAR].units})")
    plt.legend()
    plt.tight_layout()


In [ ]:
# ==================================================
# 5. Example plots
# ==================================================

plot_global("D_Tg")
plot_global("D_Focean")
plt.show()